# Análisis SQL de transacciones financieras

## Objetivo

Analizar transacciones bancarias y el comportamiento de usuarios de una fintech ficticia, utilizando consultas SQL sobre una base relacional (SQLite), para:

- entender cómo usan las tarjetas los clientes,
- identificar patrones por segmento y categoría de comercio,
- y detectar posibles anomalías (incluyendo transacciones marcadas como fraude).

El foco de este proyecto es demostrar **dominio de SQL sobre un modelo relacional realista**, más que construir modelos de machine learning.

---

## Dataset

**Fuente:** *Financial Transactions Dataset* (Kaggle)  
**Tablas utilizadas:**

- `users_data`
- `cards_data`
- `transactions_data`
- `mcc_codes`
- `train_fraud_labels`

Para este notebook se trabaja con una **muestra de 99,999 transacciones** y **2,000 usuarios**, suficiente para simular el volumen de una fintech en producción sin comprometer recursos de ejecución.



## 1. Conociendo el dataset

Antes de escribir una sola consulta SQL, es clave entender **qué representa cada tabla** y qué rol juega en el modelo de negocio.

### 1.1 Tablas principales

- 🧍 **`users_data`**  
  Contiene información del cliente:
  - identificador único (`id`),
  - edad actual y edad de retiro,
  - datos demográficos (género, dirección, coordenadas aproximadas),
  - indicadores financieros (ingreso anual, deuda total, puntaje de crédito, número de tarjetas, etc.).  

  Esta tabla nos permite analizar el comportamiento transaccional por **perfil de usuario**.

---

- 💳 **`cards_data`**  
  Describe las tarjetas emitidas a cada cliente:
  - `id` de tarjeta y `client_id` (relación con `users_data`),
  - marca (Visa/Mastercard), tipo (crédito/débito/prepago),
  - límite de crédito,
  - fechas de apertura y cambios de PIN,  
  - presencia de chip y número de tarjetas emitidas.

  Se utiliza para estudiar **qué tipo de tarjeta** está detrás de cada transacción.

---

- 🛒 **`transactions_data`**  
  Es el corazón del proyecto:
  - `id` de transacción,
  - fecha y hora,
  - `client_id` y `card_id`,
  - monto de la operación (en formato texto con símbolo de `$`),
  - identificador del comercio (`merchant_id`),
  - ciudad, estado y código postal,
  - código MCC (`mcc`) que clasifica el tipo de comercio.

  A partir de esta tabla calculamos:
  - volumen diario,
  - ticket promedio,
  - recurrencia por cliente,
  - actividad por categoría y por región.

---

- 📂 **`mcc_codes`**  
  Diccionario de **códigos MCC → categoría de comercio**.  
  Ejemplos: supermercados, gasolineras, restaurantes, tiendas departamentales, etc.  
  Nos permite traducir transacciones de “código numérico” a **categorías de negocio entendibles por cualquier stakeholder**.

---

- 🚨 **`train_fraud_labels`**  
  Tabla con etiquetas de fraude a nivel de `transaction_id`.  
  Cada transacción se clasifica como:
  - `Yes` → transacción fraudulenta,
  - `No` → transacción legítima.

  Para esta muestra:
  - se cruzan 67,231 etiquetas con la tabla de transacciones,
  - de las cuales **107 transacciones** están marcadas como fraude (≈0.1 % del total).

  Esto habilita un módulo específico de **análisis de fraude** (tasa, ticket promedio, categorías y horas de mayor riesgo).

---

En los siguientes apartados se construye un **modelo relacional en SQLite** con estas tablas y se exploran las principales métricas de negocio.


## 2. Carga y exploración inicial de las tablas

En esta sección se realiza una **primera exploración descriptiva** del dataset:

1. Carga de los archivos originales (`users_data.csv`, `cards_data.csv`, `transactions_sample.csv`, `mcc_codes.json`, `train_fraud_labels.json`).
2. Revisión del número de filas por tabla para entender la escala del problema.
3. Visualización de las primeras filas (`head()`) para validar:
   - nombres de columnas,
   - tipos de datos aproximados,
   - presencia de valores faltantes o formatos poco limpios (por ejemplo, montos con símbolo `$`).

Esta exploración inicial sirve como “control de calidad” antes de construir el modelo relacional y empezar a escribir consultas SQL de negocio.


In [1]:
import pandas as pd
import sqlite3

# --- 1. Carga de datos con muestra ligera ---

users = pd.read_csv("users_data.csv")
cards = pd.read_csv("cards_data.csv")
transactions = pd.read_csv("transactions_sample.csv")  # usamos la muestra reducida

print("Tamaños de las tablas cargadas:")
print("users:", len(users))
print("cards:", len(cards))
print("transactions (muestra):", len(transactions))

display(users.head())
display(cards.head())
display(transactions.head())



Tamaños de las tablas cargadas:
users: 2000
cards: 6146
transactions (muestra): 99999


,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


## 2. Exploración inicial del dataset

Antes de construir el modelo relacional y comenzar con las consultas SQL, se realiza una **inspección exploratoria** de las principales tablas. Esta etapa funciona como una verificación preliminar de calidad de datos y permite entender la estructura general del dataset.

---

## 🔍 2.1 Carga de las tablas y verificación de tamaños

Se cargaron los siguientes archivos originales:

- `users_data.csv`  
- `cards_data.csv`  
- `transactions_sample.csv` (muestra reducida de ~100k transacciones para evitar problemas de memoria)

Luego se verificó el tamaño de cada tabla.

### **Hallazgos:**

- **users_data:** 2,000 usuarios  
- **cards_data:** 6,146 tarjetas  
- **transactions_sample:** 99,999 transacciones  

📌 *La relación tarjetas/usuarios indica que muchos clientes poseen más de una tarjeta, lo cual es típico en una fintech con productos débito/crédito.*

📌 *Aunque usamos una muestra de transacciones, el tamaño sigue siendo suficiente para observar patrones reales de comportamiento.*

---

## 👤 2.2 Exploración de la tabla `users_data`

Esta tabla contiene información demográfica y financiera del cliente:

- Edad actual y edad de retiro  
- Año y mes de nacimiento  
- Género  
- Dirección completa  
- Ingresos (`per_capita_income`, `yearly_income`)  
- Nivel de deuda (`total_debt`)  
- Indicador crediticio (`credit_score`)  
- Número de tarjetas activas (`num_credit_cards`)

### **Insights preliminares:**

- Los ingresos y deudas vienen en formato texto con `$`, lo que indica necesidad de limpieza.  
- Existe una amplia variabilidad entre los ingresos y los niveles de deuda.  
- La distribución de género es relativamente equilibrada.  
- `latitude` y `longitude` parecen mapear estados/ciudades.

---

## 💳 2.3 Exploración de la tabla `cards_data`

Incluye información operativa de cada tarjeta:

- Marca (`Visa`, `Mastercard`, etc.)  
- Tipo de producto (débito, crédito, prepago)  
- Fechas clave (apertura, expiración, cambio de PIN)  
- Límite de crédito  
- Indicador de chip  

### **Comentarios relevantes:**

- Existen tarjetas de muy bajo límite (ej. $28 USD) y otras de límites altos → útil para segmentar perfiles de riesgo.  
- Gran variación en el año de apertura, lo que sugiere tanto clientes nuevos como antiguos.

---

## 🧾 2.4 Exploración de la tabla `transactions_sample`

Contiene las principales operaciones financieras:

- ID de transacción  
- Fecha y hora  
- Cliente y tarjeta asociada  
- Monto (incluye valores negativos por devoluciones)  
- Ciudad/estado del comercio  
- Categoría MCC  

### **Observaciones clave:**

- Los montos requieren limpieza para eliminar símbolos y convertirlos a formato numérico.  
- La granularidad temporal (a nivel de minuto) es suficiente para detectar picos de actividad o comportamientos inusuales.  
- Existen valores nulos en la columna `errors`, posiblemente un campo reservado o poco utilizado.

---

## 🧭 Conclusión de la exploración inicial

La estructura del dataset es consistente y adecuada para análisis de comportamiento y anomalías.  
Este análisis preliminar confirma que:

- Las claves (`id`, `client_id`, `card_id`) permitirán construir un modelo relacional robusto.  
- El volumen de datos de la muestra es suficiente para análisis SQL.  
- Se requiere limpieza de variables monetarias y estandarización de fechas.  

Con esto, avanzamos al siguiente paso:  
👉 **Construcción del modelo relacional en SQLite y ejecución de consultas SQL de negocio.**


## 3. Construcción del modelo relacional en SQLite

En esta etapa convertimos los DataFrames previamente cargados en una **base de datos relacional SQLite**, lo que nos permitirá ejecutar consultas SQL complejas de forma eficiente y ordenada.

Para evitar problemas de rendimiento y bloqueo de archivos (especialmente con datasets grandes), se opta por crear la base de datos **en memoria (`:memory:`)**.  

Esto ofrece varias ventajas:

### 🔹 ¿Por qué usar SQLite en memoria?
- **Mayor velocidad:** las consultas se ejecutan completamente en RAM.  
- **Evita archivos corruptos:** ideal cuando se crean y eliminan tablas repetidamente.  
- **Entorno limpio:** cada ejecución parte de una base vacía — perfecto para notebooks.  
- **Pesa menos:** no genera `.db` en disco.

### 🔹 ¿Qué tablas se crean?
- `users`  
- `cards`  
- `transactions`  

Cada una se inserta respetando su estructura original, sin índices adicionales para mantener simplicidad en esta primera versión del modelo.

---

Al finalizar este paso, contamos con una base lista para escribir consultas SQL que analicen volumen, ticket promedio, comportamiento por categoría, segmentos de usuarios, y más.

A continuación se ejecuta el código que construye la base en memoria.


In [2]:
import sqlite3

conn = sqlite3.connect(":memory:")

users.to_sql("users", conn, if_exists="replace", index=False)
cards.to_sql("cards", conn, if_exists="replace", index=False)
transactions.to_sql("transactions", conn, if_exists="replace", index=False)

print("Tablas 'users', 'cards' y 'transactions' creadas en la base SQLite en memoria.")

Tablas 'users', 'cards' y 'transactions' creadas en la base SQLite en memoria.


## 4. Verificación del modelo relacional con un JOIN de tres tablas

Antes de avanzar a consultas más complejas, es fundamental validar que la base SQLite quedó correctamente construida y que las relaciones entre tablas funcionan como deberían. Para ello, realizamos un **JOIN entre `transactions`, `users` y `cards`**, que son los tres bloques centrales del modelo.

Este primer JOIN sirve como prueba de integración y permite confirmar:

### 🔹 Que las claves coinciden correctamente:
- `transactions.client_id` ↔ `users.id`  
- `transactions.card_id` ↔ `cards.id`

### 🔹 Que los datos fluyen de forma coherente:
- Cada transacción trae información del cliente (edad, género).  
- Cada transacción también incorpora los metadatos de la tarjeta (marca, tipo).  

### 🔹 Que no existen duplicados inesperados ni pérdidas de información.

En resumen, este paso es el equivalente a un **“smoke test”** del modelo relacional: verificamos que el sistema responde correctamente y que todas las conexiones entre tablas son sólidas.

A continuación ejecutamos un JOIN de prueba y visualizamos las primeras filas.


In [3]:
# --- 3. Probar una consulta SQL con JOIN de 3 tablas ---

query = """
SELECT 
    t.id            AS transaction_id,
    t.date,
    t.amount,
    u.id            AS user_id,
    u.current_age,
    u.gender,
    c.id            AS card_id,
    c.card_brand,
    c.card_type
FROM transactions t
JOIN users u
    ON t.client_id = u.id
JOIN cards c
    ON t.card_id = c.id
LIMIT 10;
"""

ejemplo = pd.read_sql_query(query, conn)
display(ejemplo)

,transaction_id,date,amount,user_id,current_age,gender,card_id,card_brand,card_type
0,7475327,2010-01-01 00:01:00,$-77.00,1556,30,Female,2972,Mastercard,Debit (Prepaid)
1,7475328,2010-01-01 00:02:00,$14.57,561,48,Male,4575,Mastercard,Credit
2,7475329,2010-01-01 00:02:00,$80.00,1129,49,Male,102,Mastercard,Debit
3,7475331,2010-01-01 00:05:00,$200.00,430,52,Female,2860,Mastercard,Debit
4,7475332,2010-01-01 00:06:00,$46.41,848,51,Male,3915,Visa,Debit
5,7475333,2010-01-01 00:07:00,$4.81,1807,47,Female,165,Mastercard,Debit (Prepaid)
6,7475334,2010-01-01 00:09:00,$77.00,1556,30,Female,2972,Mastercard,Debit (Prepaid)
7,7475335,2010-01-01 00:14:00,$26.46,1684,56,Male,2140,Mastercard,Debit (Prepaid)
8,7475336,2010-01-01 00:21:00,$261.58,335,46,Female,5131,Visa,Debit
9,7475337,2010-01-01 00:21:00,$10.74,351,91,Female,1112,Mastercard,Debit (Prepaid)


### ✔️ Interpretación del primer JOIN entre `transactions`, `users` y `cards`

El resultado confirma que **nuestro modelo relacional está funcionando correctamente**.  
A través del JOIN, podemos observar cómo cada transacción se enriquece con datos del usuario y de la tarjeta utilizada.

#### 🔍 Validaciones clave

- **Integridad del join**:  
  Cada fila muestra información consistente entre las tres tablas.  
  Ejemplo: la transacción `7475327` pertenece al usuario `1556` y fue realizada con la tarjeta `2972`.  
  Esa relación aparece de forma coherente tanto en `users` como en `cards`.

- **Enriquecimiento de información**:  
  Ahora cada transacción no es solo un monto y un timestamp, sino que incluye atributos como:
  - Edad y género del usuario.  
  - Marca de la tarjeta (Visa, Mastercard).  
  - Tipo de tarjeta (Débito, Crédito, Prepaid).

- **Formateo del monto**:  
  Se observan montos con formato de texto que incluyen el símbolo `$`.  
  Esto se corregirá más adelante antes de realizar cálculos agregados.

- **Estructura temporal**:  
  Las fechas aparecen en formato `YYYY-MM-DD HH:MM:SS`, lo cual facilita futuras consultas de:
  - volumen por día  
  - patrones por hora  
  - análisis temporal de fraude  

#### 🎯 Conclusión

El modelo relacional (SQLite en memoria) está correctamente montado y los joins funcionan tal como se espera.  
Con esta verificación superada, ya estamos listos para avanzar a las **consultas SQL de negocio**, donde analizaremos:

- volumen de transacciones,  
- ticket promedio,  
- comportamiento por categorías,  
- actividad por usuario,  
- y posteriormente, el análisis de fraude.

Este es el punto en el que la base se vuelve “analítica”: conectamos datos aislados en un ecosistema coherente que permite responder preguntas de negocio reales.


### 📊 Análisis de volumen diario de transacciones

Con esta consulta realizamos el **primer análisis descriptivo de negocio** utilizando únicamente SQL.

El objetivo es conocer **cuántas transacciones se realizan por día**, lo cual permite:

- detectar patrones de actividad,
- evaluar estacionalidad o picos inusuales,
- preparar el terreno para detectar anomalías o posibles fraudes.



In [4]:
query = """
SELECT 
    DATE(t.date) AS transaction_date,
    COUNT(*) AS total_transactions
FROM transactions t
GROUP BY DATE(t.date)
ORDER BY DATE(t.date)
LIMIT 30;
"""

df_volumen = pd.read_sql_query(query, conn)
display(df_volumen)


,transaction_date,total_transactions
0,2010-01-01,3463
1,2010-01-02,2989
2,2010-01-03,3311
3,2010-01-04,3244
4,2010-01-05,3330
5,2010-01-06,3365
6,2010-01-07,3346
7,2010-01-08,3016
8,2010-01-09,3102
9,2010-01-10,3416


### 📈 Interpretación del volumen diario de transacciones

El resultado muestra el número total de transacciones registradas por día durante los primeros días del dataset. Para este periodo inicial (enero de 2010), se observan entre **2,900 y 3,400 transacciones diarias**, con variaciones moderadas.

#### 🧐 ¿Qué indican estos números?

- El volumen diario es **estable**, sin picos anormales ni días con caídas drásticas.
- Las variaciones naturales entre días (≈ ±200 transacciones) sugieren un comportamiento normal del sistema.
- No se presenta ningún día con un valor extremadamente bajo, lo cual indica:
  - ausencia de fallas de captura,
  - integridad correcta de la tabla,
  - y continuidad en el registro de actividad.

#### 📌 Por qué es relevante este análisis:

- Sirve como **línea base** para comparar otros análisis, por ejemplo:
  - picos de fraude,
  - comportamiento por hora o por categoría,
  - días con actividad atípica.
- También permite validar que la migración de CSV a SQLite no perdió información.

#### 🧭 Insight inicial

El comportamiento estable sugiere una plataforma con tráfico constante y homogéneo. Más adelante podremos contrastar estos valores con:

- días de mayor proporción de fraude,
- horas más riesgosas,
- categorías con variaciones abruptas.

Este análisis abre la puerta a detectar anomalías con mayor precisión.


In [5]:
query = """
SELECT 
    COUNT(*) AS total_transactions,
    SUM(
        REPLACE(REPLACE(t.amount, '$', ''), ',', '') * 1.0
    ) AS total_amount,
    AVG(
        REPLACE(REPLACE(t.amount, '$', ''), ',', '') * 1.0
    ) AS avg_ticket
FROM transactions t;
"""

df_kpis = pd.read_sql_query(query, conn)
display(df_kpis)


,total_transactions,total_amount,avg_ticket
0,99999,4312077.84,43.12121


### 💹 KPIs generales del comportamiento financiero

A partir de la tabla `transactions`, se calcularon tres métricas fundamentales para comprender la escala económica del dataset:

#### 📌 1. Volumen total de transacciones  
La muestra contiene **99,999 transacciones**, lo cual confirma un tamaño suficiente para análisis estadísticos fiables y para detectar patrones, incluso en eventos poco frecuentes (como el fraude).

#### 📌 2. Monto total transaccionado  
El total acumulado asciende a **\$4,312,077.84 USD**.  
Este valor consolidado permite dimensionar la magnitud económica del sistema y estimar el impacto potencial de anomalías —por ejemplo, pérdidas asociadas a fraude.

#### 📌 3. Ticket promedio  
El ticket promedio es de **\$43.12 USD**.  
Este valor sirve como referencia para identificar transacciones inusualmente altas o bajas, que más adelante podremos analizar como posibles outliers.

---

### 🧐 ¿Qué nos dice este bloque de KPIs?

- El sistema maneja un **volumen alto y estable**, típico de una fintech con transacciones recurrentes.
- El monto promedio sugiere que la mayoría de las operaciones corresponden a compras de consumo cotidiano.
- El total anualizado (solo con la muestra) apunta a un sistema con **movimiento financiero significativo**, donde incluso pequeñas variaciones en fraude pueden representar pérdidas importantes.

---

### 🎯 Insight inicial

Estos KPIs son el “tablero principal” del análisis financiero: sirven como punto de partida para comparar subgrupos de datos (por categoría, usuario, hora, etc.) y para detectar zonas donde los valores se desvían del comportamiento típico.

A partir de aquí, el proyecto puede avanzar hacia análisis más específicos como:

- ticket promedio por merchant category (MCC),  
- patrones por región,  
- montos atípicos,  
- horas o días con mayor riesgo.  


### 👥 Actividad diaria de usuarios (Active Users)

En este paso analizamos cuántos **usuarios únicos realizan al menos una transacción por día**.  
Para ello, se utiliza la métrica:

- **Active Users (DAU):** número de `client_id` distintos que emitieron una transacción en una fecha dada.

Esta métrica es crítica para entender el nivel de uso real del sistema financiero y detectar picos o caídas inusuales en la actividad.

---

### 🔍 ¿Por qué es importante esta métrica?

- Mide la **participación real** de los clientes, no solo el volumen de transacciones.
- Permite identificar **comportamientos estacionales**: días con más actividad, posibles festivos, fines de semana, etc.
- Cambios abruptos en el número de usuarios activos pueden sugerir:
  - fallos del sistema,
  - campañas o promociones,
  - eventos externos,
  - o comportamientos anómalos.

---

### 📊 Resultados iniciales (primeros 30 días)

La tabla resultante muestra los primeros 30 días registrados, con su respectivo número de usuarios activos.  
Esto nos permite observar:

- variabilidad diaria normal en la actividad,
- estabilidad o fluctuaciones inusuales,
- niveles base de uso del sistema.

Esta información es esencial antes de avanzar hacia análisis más complejos como cohortes, recurrencia o segmentación por grupos de clientes.

---

### 🎯 Insight inicial

Los valores reportados muestran una base sólida de usuarios activos diarios, lo que indica un ecosistema financiero con uso recurrente y comportamiento relativamente estable.

Este dato también será clave más adelante para evaluar:
- qué tan distribuido está el uso del sistema,
- si los incidentes de fraude están concentrados en pocos usuarios o distribuidos,
- y cómo se comporta el ticket promedio en función de la actividad diaria.



In [6]:
query = """
SELECT 
    DATE(t.date) AS transaction_date,
    COUNT(DISTINCT t.client_id) AS active_users
FROM transactions t
GROUP BY DATE(t.date)
ORDER BY 1
LIMIT 30;
"""

df_activos = pd.read_sql_query(query, conn)
display(df_activos)


,transaction_date,active_users
0,2010-01-01,978
1,2010-01-02,954
2,2010-01-03,983
3,2010-01-04,988
4,2010-01-05,980
5,2010-01-06,992
6,2010-01-07,984
7,2010-01-08,973
8,2010-01-09,956
9,2010-01-10,988


### 📆 Usuarios activos por día – Interpretación de resultados

La consulta devuelve el número de **usuarios únicos** (`client_id`) que realizaron al menos una transacción en cada día del período analizado.  

Los primeros registros muestran valores que oscilan entre **950 y 990 usuarios activos diarios**, lo cual sugiere:

---

### 🔎 1. Estabilidad operativa  
La actividad diaria de usuarios se mantiene dentro de un rango estrecho.  
Esto indica que la plataforma:

- tiene un **uso constante**,  
- no presenta días con caídas abruptas,  
- y no depende de picos irregulares para generar transacciones.

Esta estabilidad es típica de un sistema financiero con usuarios que utilizan sus tarjetas de forma cotidiana.

---

### 🔎 2. Patrones saludables de uso  
No se observan saltos dramáticos de un día a otro.  
Esto es señal de:

- **comportamiento orgánico**,  
- alta recurrencia,  
- y una base de usuarios que permanece activa de forma consistente.

En proyectos reales de fintech, este tipo de patrón suele correlacionar con:

- clientes con pagos recurrentes,  
- hábitos de consumo regulares,  
- y buena retención.

---

### 🔎 3. Oportunidad para análisis avanzado  
Con esta estabilidad como punto de partida, se habilitan futuros análisis como:

- retención semanal (WAU / DAU),  
- cohortes por mes de registro,  
- estacionalidad por día de la semana,  
- correlación entre actividad diaria y ticket promedio,  
- detección de anomalías si algún día se aleja del rango.

---

### 📌 Insight rápido

Durante los primeros 14 días observados, el número de usuarios activos diarios:

- se mantiene entre **954 y 994**,  
- muestra variaciones naturales pero pequeñas,  
- y sugiere una base de usuarios **sólida y participativa**.

Este comportamiento es un buen indicador antes de avanzar hacia el análisis de fraude, ticket promedio y categorías de gasto.



In [7]:
import json
import pandas as pd

# --- Cargar MCC Codes (diccionario codigo -> descripción) ---

with open("mcc_codes.json", "r") as f:
    mcc_data = json.load(f)

# Si es un dict tipo {"5499": "Grocery", ...}
if isinstance(mcc_data, dict):
    rows = [
        {"mcc_code": int(k), "merchant_category_name": v}
        for k, v in mcc_data.items()
    ]
    mcc = pd.DataFrame(rows)
# Si fuera lista de dicts, lo manejamos también
elif isinstance(mcc_data, list):
    mcc = pd.DataFrame(mcc_data)
else:
    raise ValueError("Formato inesperado en mcc_codes.json")

display(mcc.head())
print("Filas en mcc:", len(mcc))



,mcc_code,merchant_category_name
0,1711,"Heating, Plumbing, Air Conditioning Contractors"
1,3000,Steelworks
2,3001,Steel Products Manufacturing
3,3005,Miscellaneous Metal Fabrication
4,3006,Miscellaneous Fabricated Metal Products


Filas en mcc: 109


## 🗂️ 4. Carga de Tabla MCC Codes (Merchant Category Codes)

Los MCC Codes son un catálogo que asigna a cada código de comercio (mcc_code) una categoría descriptiva (merchant_category_name).
Este catálogo es clave porque:

Permite traducir los códigos MCC presentes en las transacciones.

Habilita análisis de negocio como:

Rubros con mayor volumen.

Categorías con tickets más altos.

Categorías con mayor incidencia de fraude.

Sirve como tabla catálogo para posteriores JOIN en SQL.

En esta sección se realiza:

Carga del archivo mcc_codes.json.

Identificación del formato de origen (diccionario o lista).

Normalización a un DataFrame estándar con dos columnas:

mcc_code

merchant_category_name

💡 Este paso prepara la base categórica para enriquecer el análisis transaccional.

## Inserción del catálogo MCC Codes en la base SQLite
Una vez transformado el archivo mcc_codes.json en un DataFrame limpio y estandarizado, el siguiente paso es integrarlo al modelo relacional.
Esto es fundamental porque:

Permite enriquecer las transacciones con descripciones de categoría.

Habilita análisis más interpretables (no solo códigos numéricos).

Facilita los JOIN entre transacciones y el catálogo.

En este paso enviamos la tabla mcc_codes directamente a la base SQLite en memoria utilizando to_sql().

📌 Importante: usamos if_exists="replace" para garantizar que no se acumulen versiones previas.

In [8]:
mcc.to_sql("mcc_codes", conn, if_exists="replace", index=False)
print("Tabla mcc_codes creada en la base.")


Tabla mcc_codes creada en la base.


La tabla mcc_codes queda correctamente almacenada en la base SQLite, y desde este punto puede ser utilizada en consultas SQL para análisis por categoría de comercio.

Esto completa el set de tablas relacionales que utilizaremos durante el proyecto.

## 6. Análisis por categoría de comercio (Top 10 MCC por volumen)

Hasta ahora hemos analizado el comportamiento global de las transacciones y la actividad por día.  
El siguiente paso es responder una pregunta muy típica en proyectos financieros:

> **¿En qué tipos de comercios se concentran más las transacciones?**

Para ello, unimos la tabla de transacciones (`transactions`) con el catálogo de categorías (`mcc_codes`) y calculamos el **número total de transacciones por categoría de comercio** (`merchant_category_name`).

Puntos clave de la consulta:

- `JOIN mcc_codes`  
  Enlazamos cada transacción con su categoría MCC descriptiva.

- `COUNT(*) AS total_transactions`  
  Contamos cuántas operaciones se registran en cada categoría.

- `ORDER BY total_transactions DESC`  
  Ordenamos de mayor a menor para identificar las categorías más utilizadas.

- `LIMIT 10`  
  Nos quedamos con el **Top 10** para una lectura rápida y enfocada.

A continuación se muestra la consulta y el resultado obtenido.


In [9]:
query = """
SELECT 
    m.merchant_category_name,
    COUNT(*) AS total_transactions
FROM transactions t
JOIN mcc_codes m
    ON CAST(t.mcc AS INTEGER) = m.mcc_code
GROUP BY m.merchant_category_name
ORDER BY total_transactions DESC
LIMIT 10;
"""

df_top_mcc = pd.read_sql_query(query, conn)
display(df_top_mcc)


,merchant_category_name,total_transactions
0,"Grocery Stores, Supermarkets",12025
1,Miscellaneous Food Stores,11253
2,Service Stations,10961
3,Eating Places and Restaurants,7544
4,Drug Stores and Pharmacies,5704
5,Money Transfer,4563
6,Wholesale Clubs,4542
7,Tolls and Bridge Fees,4510
8,Taxicabs and Limousines,3877
9,Fast Food Restaurants,3761


### 📊 Interpretación: Top 10 categorías por número de transacciones

El resultado muestra las **10 categorías de comercio con mayor volumen de transacciones**.  
Entre las más relevantes aparecen:

- **Grocery Stores, Supermarkets**  
- **Miscellaneous Food Stores**  
- **Service Stations**  
- **Eating Places and Restaurants**  
- **Drug Stores and Pharmacies**  
- **Money Transfer**, **Wholesale Clubs**, **Tolls and Bridge Fees**, **Taxicabs and Limousines**, **Fast Food Restaurants**.

#### 🧠 ¿Qué nos dice este patrón?

1. **Predominio del consumo cotidiano**  
   Supermercados, tiendas de comida y restaurantes concentran una gran parte del volumen.  
   Esto sugiere que las tarjetas se usan principalmente para gastos diarios y recurrentes.

2. **Relevancia de transporte y servicios**  
   Categorías como gasolineras, taxis y peajes indican un uso intenso en movilidad y servicios complementarios.

3. **Canales de dinero y clubes mayoristas**  
   `Money Transfer` y `Wholesale Clubs` apuntan a operaciones de envíos de dinero y compras más grandes o planificadas.

#### 🎯 Valor para el negocio

Este ranking permite a una fintech:

- Identificar los **segmentos clave de gasto** de sus usuarios.
- Detectar en qué rubros tiene más sentido:
  - lanzar promociones,  
  - negociar alianzas,  
  - o monitorear más de cerca el riesgo de fraude.

Más adelante, este análisis se podrá cruzar con:
- ticket promedio por categoría,  
- tasa de fraude por MCC,  
- y patrones por rango de edad o zona geográfica.

En resumen, este Top 10 revela que la mayor parte del movimiento se concentra en **gastos de consumo básico y movilidad**, lo cual encaja con el comportamiento esperado de una base amplia de clientes.


## 📊 Análisis de ticket promedio por categoría (MCC)

En esta etapa profundizamos en qué categorías de comercios generan tickets promedio más altos, una métrica útil para entender el tipo de gasto que predomina en los clientes y detectar patrones atípicos.

Para ello:

Convertimos los montos textuales a valores numéricos (quitando $ y comas).

Calculamos el ticket promedio por categoría.

Incluimos solo categorías con más de 3,000 transacciones, garantizando que el análisis se base en suficiente volumen.

Ordenamos de mayor a menor ticket promedio para identificar los segmentos “premium”.

Este análisis es clave para reconocer:

categorías con gasto alto (mayor riesgo financiero),

patrones de consumo diferenciados,

posibles focos de fraude ligados a montos elevados.

In [10]:
query = """
SELECT 
    m.merchant_category_name,
    AVG(REPLACE(REPLACE(t.amount, '$', ''), ',', '') * 1.0) AS avg_ticket,
    COUNT(*) AS num_transactions
FROM transactions t
JOIN mcc_codes m
    ON CAST(t.mcc AS INTEGER) = m.mcc_code
GROUP BY m.merchant_category_name
HAVING COUNT(*) > 3000
ORDER BY avg_ticket DESC
LIMIT 10;
"""

df_ticket_cat = pd.read_sql_query(query, conn)
display(df_ticket_cat)


,merchant_category_name,avg_ticket,num_transactions
0,Money Transfer,90.361604,4563
1,Wholesale Clubs,65.306953,4542
2,Department Stores,56.528643,3545
3,Automotive Service Shops,51.364352,3504
4,Drug Stores and Pharmacies,45.877596,5704
5,Tolls and Bridge Fees,35.010253,4510
6,Eating Places and Restaurants,27.095138,7544
7,Fast Food Restaurants,26.314823,3761
8,"Grocery Stores, Supermarkets",25.703552,12025
9,Taxicabs and Limousines,22.931751,3877


## 🧠 Interpretación de resultados

El resultado confirma tendencias muy marcadas:

💸 Money Transfer encabeza el ranking con un ticket promedio elevado, reflejando transacciones de alto valor típicas de envíos de dinero.

Wholesale Clubs y Department Stores mantienen tickets altos debido a compras voluminosas o de mayor valor unitario.

Categorías como Eating Places o Fast Food aparecen al final del ranking: montos bajos pero con alta frecuencia.

Este tipo de análisis permite entender qué comercios concentran mayor exposición monetaria y es especialmente valioso para sistemas antifraude, segmentación de clientes y análisis de rentabilidad por vertical.

## 📊 3.4. Clientes con mayor recurrencia

En esta sección identificamos a los usuarios más activos dentro del periodo analizado, ordenando por la cantidad total de transacciones realizadas. Este análisis permite:

Detectar clientes altamente comprometidos con el servicio.

Observar posibles comportamientos atípicos (usuarios con volúmenes inusualmente altos).

Entender qué tan distribuida está la actividad dentro de la base de clientes.

La consulta agrupa las transacciones por client_id y devuelve a los 20 usuarios con mayor número de movimientos.

In [11]:
query = """
SELECT 
    t.client_id AS user_id,
    COUNT(*) AS num_transactions
FROM transactions t
GROUP BY t.client_id
ORDER BY num_transactions DESC
LIMIT 20;
"""

df_recurrencia = pd.read_sql_query(query, conn)
display(df_recurrencia)


,user_id,num_transactions
0,909,391
1,1098,382
2,1963,362
3,1776,322
4,96,314
5,1888,311
6,1696,304
7,114,295
8,285,285
9,1575,281


Los datos muestran que:

El cliente más activo registra 391 transacciones, seguido de otros con >350 movimientos.

Existe un pequeño grupo de usuarios que concentran una fracción considerable del volumen total, lo cual puede indicar:

Clientes heavy-users con hábitos financieros muy constantes.

Posibles casos interesantes para programas de lealtad o análisis de riesgo.

Potenciales señales para monitorear patrones inusuales, aunque por sí solos no indican fraude.

En general, se observa una cola larga de actividad, donde unos pocos clientes realizan muchas más transacciones que la mayoría — patrón común en servicios financieros y comercios digitales.

## 📊 3.5. ¿Qué tipo de categorías consumen usuarios de mayor edad?

Antes de ejecutar la consulta, vale la pena entender qué estamos buscando exactamente.

Las finanzas personales y los patrones de consumo suelen variar de forma notable entre grupos de edad. Algunas categorías tienden a concentrar transacciones de usuarios jóvenes (ej. apps digitales), mientras que otras suelen atraer a usuarios de mayor edad (ej. clubes mayoristas, servicios médicos o transporte).

En esta sección analizamos:

Promedio de edad de los clientes que compran en cada categoría MCC.

Volumen mínimo de 2,000 transacciones por categoría (para evitar sesgos por categorías demasiado pequeñas).

Ranking descendente según la edad promedio.

Esto permite identificar:

categorías con usuarios mayores,

segmentos con alto poder adquisitivo,

oportunidades de marketing o programas de fidelización según edad.

In [12]:
query = """
SELECT 
    m.merchant_category_name,
    ROUND(AVG(u.current_age), 1) AS avg_age,
    COUNT(*) AS num_transactions
FROM transactions t
JOIN users u
    ON t.client_id = u.id
JOIN mcc_codes m
    ON CAST(t.mcc AS INTEGER) = m.mcc_code
GROUP BY m.merchant_category_name
HAVING num_transactions > 2000
ORDER BY avg_age DESC
LIMIT 10;
"""

df_age_cat = pd.read_sql_query(query, conn)
display(df_age_cat)


,merchant_category_name,avg_age,num_transactions
0,Taxicabs and Limousines,59.0,3877
1,Wholesale Clubs,55.5,4542
2,"Grocery Stores, Supermarkets",55.4,12025
3,Eating Places and Restaurants,55.3,7544
4,Department Stores,55.3,3545
5,Fast Food Restaurants,55.2,3761
6,Automotive Service Shops,55.1,3504
7,Money Transfer,55.0,4563
8,Drug Stores and Pharmacies,54.9,5704
9,Service Stations,54.6,10961


Los resultados muestran que las categorías con mayor edad promedio están relacionadas con:

Transporte tradicional (Taxi & Limousines)

Clubes mayoristas (Wholesale Clubs)

Supermercados

Restaurantes

Tiendas departamentales

Este patrón sugiere que usuarios de mayor edad concentran su gasto en consumo esencial, movilidad y compras de volumen, lo cual es consistente con comportamientos financieros típicos de adultos mayores con estabilidad económica.

## 📌 4.2. Detección de anomalías por monto (outliers financieros)

En este apartado buscamos transacciones atípicamente altas, que pueden representar:

consumos inusuales del cliente,

errores en el registro del comercio,

o potenciales señales de fraude.

Para identificarlas:

Convertimos el campo amount (que viene como texto con $ y comas) a formato numérico dentro de la consulta SQL.

Filtramos únicamente aquellas transacciones con monto mayor a 2000 USD.

Enlazamos los códigos MCC para entender el tipo de comercio asociado.

Ordenamos de mayor a menor para ver primero las transacciones más elevadas.

Este filtrado es una primera aproximación rápida antes de trabajar con modelos estadísticos o machine learning.

In [13]:
query = """
SELECT 
    t.id AS transaction_id,
    t.date,
    t.client_id,
    t.card_id,
    t.amount,
    m.merchant_category_name
FROM transactions t
LEFT JOIN mcc_codes m
    ON CAST(t.mcc AS INTEGER) = m.mcc_code
WHERE 
    REPLACE(REPLACE(t.amount, '$', ''), ',', '') * 1.0 > 2000
ORDER BY 
    REPLACE(REPLACE(t.amount, '$', ''), ',', '') * 1.0 DESC
LIMIT 20;
"""

df_anom = pd.read_sql_query(query, conn)
display(df_anom)


,transaction_id,date,client_id,card_id,amount,merchant_category_name
0,7512096,2010-01-10 11:50:00,1340,2954,$2807.05,Cruise Lines
1,7517440,2010-01-11 16:50:00,1487,4946,$2680.80,Wholesale Clubs
2,7576631,2010-01-26 21:22:00,708,5165,$2114.99,Antique Shops


## 📊 Interpretación de resultados

A partir de las transacciones filtradas:

Los montos superan los 2000 USD, lo cual es atípico considerando la distribución general del dataset.

Las categorías asociadas (como Cruise Lines, Wholesale Clubs, Antique Shops) corresponden a rubros donde naturalmente pueden aparecer consumos elevados.

Aun así, este tipo de operaciones deben monitorearse, especialmente si el cliente no suele consumir en estas categorías o si el patrón no coincide con su historial.

Este análisis abre la puerta a técnicas más avanzadas como detección de outliers por IQR, clustering o modelos supervisados de fraude.

## 📊 Análisis por Rango de Edad
¿Cómo cambia el comportamiento transaccional según la etapa de vida?

En esta sección exploramos si la edad influye en el volumen de transacciones y en el ticket promedio.
Para lograrlo:

Segmentamos a los usuarios en rangos de edad claros y operativos:
18–29, 30–39, 40–49, 50–59, 60+.

Integramos la tabla de usuarios con la de transacciones mediante un JOIN.

Calculamos por cada grupo:

Número total de transacciones.

Ticket promedio (normalizando los valores monetarios que vienen con símbolo $).

Este análisis ayuda a entender qué edades concentran la mayor actividad financiera y qué segmentos gastan más por operación, información clave para marketing, productos personalizados o modelos de riesgo.

In [14]:
query = """
SELECT 
    CASE 
        WHEN u.current_age < 30 THEN '18-29'
        WHEN u.current_age BETWEEN 30 AND 39 THEN '30-39'
        WHEN u.current_age BETWEEN 40 AND 49 THEN '40-49'
        WHEN u.current_age BETWEEN 50 AND 59 THEN '50-59'
        ELSE '60+'
    END AS age_range,
    COUNT(*) AS total_transactions,
    AVG(REPLACE(REPLACE(t.amount, '$', ''), ',', '') * 1.0) AS avg_ticket
FROM transactions t
JOIN users u
    ON t.client_id = u.id
GROUP BY age_range
ORDER BY age_range;
"""

df_ages = pd.read_sql_query(query, conn)
display(df_ages)


,age_range,total_transactions,avg_ticket
0,18-29,294,30.029456
1,30-39,15723,44.307468
2,40-49,25302,41.638701
3,50-59,24634,44.497233
4,60+,34046,42.792564


## 🔍 Interpretación

✔ Los adultos mayores (60+) y los segmentos de 40–59 años concentran la mayor actividad transaccional.
Esto sugiere un patrón consistente con población económicamente estable y con hábitos financieros maduros.

✔ El ticket promedio más alto aparece en los grupos de 50–59 y 30–39 años.
Ambos segmentos suelen representar etapas de mayor ingreso disponible o gasto consolidado.

✔ El segmento 18–29 destaca por baja participación.
Es probable que en esta etapa se utilicen menos tarjetas o se realicen gastos de menor monto.

En conjunto, esta distribución es útil para diseñar estrategias de segmentación, programas de lealtad y análisis de riesgo diferenciados por edad.

## 🔍 4. Exploración inicial de fraud labels

Para complementar el análisis transaccional, incluimos la tabla con etiquetas de fraude (train_fraud_labels.json).
Estas etiquetas representan un diccionario donde cada transaction_id tiene asignado un valor "Yes" o "No" indicando si fue detectada como fraude.

Antes de integrarlo al modelo relacional, realizamos una inspección para:

Validar la estructura del archivo JSON.

Confirmar que contiene un único diccionario anidado.

Identificar el volumen total de etiquetas.

Verificar que los valores estén correctamente representados.

Este paso es clave para asegurarnos de que las etiquetas puedan cruzarse con la tabla de transacciones posteriormente.

In [15]:
import json

with open("train_fraud_labels.json", "r") as f:
    fraud_raw = json.load(f)

fraud_raw[:5] if isinstance(fraud_raw, list) else fraud_raw


{'target': {'7475327': 'No',
  '7475328': 'No',
  '7475329': 'No',
  '7475332': 'No',
  '7475333': 'No',
  '7475335': 'No',
  '7475338': 'No',
  '7475339': 'No',
  '7475340': 'No',
  '7475341': 'No',
  '7475342': 'No',
  '7475344': 'No',
  '7475345': 'No',
  '7475346': 'No',
  '7475347': 'No',
  '7475348': 'No',
  '7475349': 'No',
  '7475350': 'No',
  '7475353': 'No',
  '7475354': 'No',
  '7475357': 'No',
  '7475359': 'No',
  '7475360': 'No',
  '7475361': 'No',
  '7475362': 'No',
  '7475363': 'No',
  '7475365': 'No',
  '7475366': 'No',
  '7475367': 'No',
  '7475368': 'No',
  '7475371': 'No',
  '7475372': 'No',
  '7475377': 'No',
  '7475378': 'No',
  '7475379': 'No',
  '7475380': 'No',
  '7475382': 'No',
  '7475383': 'No',
  '7475385': 'No',
  '7475389': 'No',
  '7475390': 'No',
  '7475394': 'No',
  '7475395': 'No',
  '7475398': 'No',
  '7475400': 'No',
  '7475401': 'No',
  '7475403': 'No',
  '7475406': 'No',
  '7475410': 'No',
  '7475411': 'No',
  '7475412': 'No',
  '7475414': 'No',
  

📌 Resultado

El archivo contiene un diccionario bajo la llave "target", donde cada clave es un transaction_id y cada valor es "No" o "Yes".

Como se observa, la mayoría de las etiquetas son “No”, lo cual es normal en problemas de fraude debido a la naturaleza altamente desbalanceada del fenómeno.

Este comportamiento será relevante en etapas posteriores de análisis.

## 🔍 Conversión del archivo de etiquetas de fraude (`train_fraud_labels.json`)

El dataset incluye un archivo JSON con **etiquetas de fraude por transacción**, pero su estructura no viene en un formato tabular directo.  
En esta sección realizamos:

### ✔ 1. Lectura del archivo JSON  
Cargamos el archivo `train_fraud_labels.json`, el cual almacena un gran diccionario donde:
- La clave `"target"` contiene miles de pares `id_transacción : "Yes"/"No"`.

### ✔ 2. Conversión a DataFrame  
Transformamos ese diccionario en un DataFrame con dos columnas:

- **transaction_id** → el ID de la transacción (convertido a entero).  
- **is_fraud** → bandera binaria (`1` = fraude, `0` = legítimo).

### ✔ 3. Validación  
Mostramos:
- El total de etiquetas procesadas  
- La distribución de transacciones fraudulentas vs legítimas  
- Las primeras filas, para validar estructura y tipos

Esta tabla será crucial posteriormente para:
- unir las etiquetas al dataset principal,
- calcular tasas de fraude por categoría, usuario u horario,
- y detectar patrones anómalos en el comportamiento financiero.

---


In [16]:
import pandas as pd

# Diccionario id_transacción -> 'Yes'/'No'
target_dict = fraud_raw["target"]

fraud_rows = [
    {
        "transaction_id": int(tx_id),
        "is_fraud": 1 if label == "Yes" else 0
    }
    for tx_id, label in target_dict.items()
]

fraud = pd.DataFrame(fraud_rows)

print("Fraud labels totales:", len(fraud))
print(fraud["is_fraud"].value_counts())
fraud.head()


Fraud labels totales: 8914963
is_fraud
0    8901631
1      13332
Name: count, dtype: int64


,transaction_id,is_fraud
0,7475327,0
1,7475328,0
2,7475329,0
3,7475332,0
4,7475333,0


### 🧾 Resultados: Labels de fraude procesados correctamente

El archivo JSON fue interpretado y convertido exitosamente a formato tabular.

**Hallazgos principales:**

- Total de etiquetas importadas: **8,914,963**
- Transacciones legítimas (`is_fraud = 0`): **8,901,631**
- Transacciones fraudulentas (`is_fraud = 1`): **13,332**

Esto revela un dataset **altamente desbalanceado**, típico de escenarios reales de fraude:
- Solo el **0.15%** de las transacciones registradas en el archivo original están marcadas como fraudulentas.

El DataFrame generado tiene ahora esta estructura:

| transaction_id | is_fraud |
|----------------|----------|
| 7475327        | 0        |
| 7475328        | 0        |
| 7475329        | 0        |
| …              | …        |

Este formato permitirá más adelante **integrar la etiqueta con la tabla `transactions`** para análisis avanzados de fraude, como tasas por horario, categoría MCC o comportamiento del usuario.

---


### 🔍 Filtrado de etiquetas de fraude para coincidir con la muestra de transacciones

El archivo original de fraude contiene casi **9 millones de etiquetas**, pero en este proyecto estamos usando una **muestra reducida** de transacciones (`transactions_sample.csv`).

Para poder analizar fraude dentro de nuestra base SQLite, necesitamos:

1. **Identificar los IDs de transacción presentes en nuestra muestra.**  
   Esto se obtiene tomando los valores únicos de `transactions["id"]`.

2. **Filtrar el gran archivo de fraude** y quedarnos únicamente con las filas cuyo `transaction_id` aparece en nuestra muestra.

3. **Verificar cuántas de esas transacciones filtradas están marcadas como fraude real.**

Este paso es fundamental porque nos permite trabajar con un dataset consistente y proporcional a la muestra utilizada en los análisis previos.

---


In [17]:
# IDs de transacciones en tu muestra
sample_ids = set(transactions["id"].unique())

fraud_sample = fraud[fraud["transaction_id"].isin(sample_ids)].copy()

print("Filas de fraude que coinciden con la muestra:", len(fraud_sample))
print(fraud_sample["is_fraud"].value_counts())
fraud_sample.head()


Filas de fraude que coinciden con la muestra: 67231
is_fraud
0    67124
1      107
Name: count, dtype: int64


,transaction_id,is_fraud
0,7475327,0
1,7475328,0
2,7475329,0
3,7475332,0
4,7475333,0


### 📊 Resultados del filtrado de fraude sobre la muestra

Tras cruzar los IDs del dataset de fraude con las transacciones presentes en nuestra muestra, los resultados son:

- 🔢 **Transacciones en la muestra:** 99,999  
- 🔍 **Labels de fraude encontrados en la muestra:** 67,231  
  (es decir, no todas las transacciones del JSON estaban en la muestra, lo cual era esperado)

- 🟥 **Fraudes detectados:** 107  
- 🟩 **Transacciones legítimas:** 67,124  

Esto confirma nuevamente que el dataset es **extremadamente desbalanceado**, ya que los fraudes representan alrededor de:

\[
\text{fraud rate} = \frac{107}{67,231} \approx 0.16\%
\]

Las primeras filas del DataFrame filtrado confirman la estructura final:

| transaction_id | is_fraud |
|----------------|----------|
| 7475327        | 0        |
| 7475328        | 0        |
| 7475329        | 0        |
| …              | …        |

Con esta tabla filtrada podemos ahora:

- unirla a la tabla `transactions`,
- calcular tasas de fraude por categoría MCC,
- analizar picos horarios,
- estudiar comportamiento de usuarios fraudulentos,
- y producir alertas o insights accionables.

---


### 🗂️ Creación de la tabla `fraud_labels` en la base SQLite

Ya contamos con el DataFrame `fraud_sample`, que contiene únicamente las etiquetas de fraude correspondientes a las transacciones presentes en nuestra muestra.

El siguiente paso es **cargar esta tabla dentro de SQLite** para poder:

- realizar consultas SQL de fraude,  
- cruzarla fácilmente con `transactions`,  
- calcular tasas de fraude por categorías, horarios o usuarios,  
- y habilitar análisis posteriores de anomalías.

Usamos `to_sql()` para insertar la tabla en memoria, reemplazándola si ya existiera.


In [18]:
fraud_sample.to_sql("fraud_labels", conn, if_exists="replace", index=False)
print("Tabla fraud_labels creada en la base.")


Tabla fraud_labels creada en la base.


### ✅ Tabla `fraud_labels` creada correctamente

La tabla **`fraud_labels`** ya está disponible dentro de nuestra base SQLite.  
Esto confirma que el proceso de filtrado fue exitoso y que ahora podemos trabajar con un dataset consistente, alineado con nuestra muestra de transacciones.

A partir de este punto, podemos:

- unir `transactions` + `mcc_codes` + `users` + `fraud_labels`,
- calcular métricas como:
  - tasa de fraude por categoría MCC,
  - monto promedio de transacciones fraudulentas,
  - picos temporales de actividad sospechosa,
  - comportamiento de usuarios afectados,
- y preparar los insights finales del proyecto.

Con esta integración, entramos formalmente a la fase de **Análisis de Fraude** usando SQL.


### 🔍 Cálculo de la tasa general de fraude

Antes de analizar patrones más complejos, conviene obtener una **visión panorámica del problema**:  
¿Cuántas transacciones fueron marcadas como fraude y qué porcentaje representan dentro de la muestra?

En esta consulta:

- Contamos el **total de transacciones**.
- Sumamos cuántas tienen etiqueta de fraude (`is_fraud = 1`).
- Calculamos el **fraud_rate_pct**, expresado como porcentaje.
- Usamos `LEFT JOIN` para integrar `transactions` con `fraud_labels`, de forma que todas las transacciones se consideren (fraudulentas o no).

Este KPI funciona como punto de referencia para comparar tasas por categoría, hora del día, usuario o tipo de tarjeta.


In [19]:
query = """
SELECT 
    COUNT(*) AS total_transactions,
    SUM(CASE WHEN f.is_fraud = 1 THEN 1 ELSE 0 END) AS fraud_transactions,
    100.0 * SUM(CASE WHEN f.is_fraud = 1 THEN 1 ELSE 0 END) / COUNT(*) AS fraud_rate_pct
FROM transactions t
LEFT JOIN fraud_labels f
    ON t.id = f.transaction_id;
"""

df_fraud_rate = pd.read_sql_query(query, conn)
display(df_fraud_rate)


,total_transactions,fraud_transactions,fraud_rate_pct
0,99999,107,0.107001


### 📊 Resultado: Tasa de fraude en la muestra

Los valores obtenidos muestran:

- **Total de transacciones:** 99,999  
- **Transacciones fraudulentas:** 107  
- **Tasa de fraude:** 0.107%  

Aunque el número absoluto de transacciones fraudulentas es pequeño, su impacto no debe subestimarse.  
En escenarios reales, incluso tasas inferiores al 0.1% pueden representar pérdidas millonarias y justificar sistemas avanzados de detección.

Este resultado confirma que el dataset presenta **fraude altamente infrecuente**, lo que introduce retos clásicos en análisis financiero:

- fuerte desbalanceo de clases,  
- necesidad de métricas adicionales para entender dónde ocurre el fraude,  
- importancia de segmentar para encontrar patrones escondidos.

Con esta línea base establecida, avanzamos hacia análisis más específicos.


### 💳 Comparación entre transacciones legítimas y fraudulentas

Ahora profundizamos en el comportamiento financiero de ambos grupos:  
**¿En qué se diferencian las transacciones marcadas como fraude respecto a las legítimas?**

Para lograrlo:

- Estandarizamos el monto (`amount_clean`) eliminando el formato de texto y el símbolo `$`.
- Unificamos la etiqueta de fraude con `COALESCE`, evitando valores nulos.
- Agrupamos todas las transacciones en dos categorías:
  - **Fraud**
  - **Legit**
- Calculamos:
  - número total de transacciones,
  - ticket promedio (`avg_ticket`),
  - monto total procesado.

Este análisis permite detectar si las operaciones fraudulentas son más “caras”, más frecuentes o más dispersas, lo cual es clave para diseñar alertas específicas.


In [20]:
query = """
WITH tx AS (
    SELECT 
        t.id,
        REPLACE(REPLACE(t.amount, '$', ''), ',', '') * 1.0 AS amount_clean,
        COALESCE(f.is_fraud, 0) AS is_fraud
    FROM transactions t
    LEFT JOIN fraud_labels f
        ON t.id = f.transaction_id
)
SELECT 
    CASE WHEN is_fraud = 1 THEN 'Fraud' ELSE 'Legit' END AS label,
    COUNT(*) AS num_transactions,
    AVG(amount_clean) AS avg_ticket,
    SUM(amount_clean) AS total_amount
FROM tx
GROUP BY label;
"""

df_fraud_ticket = pd.read_sql_query(query, conn)
display(df_fraud_ticket)


,label,num_transactions,avg_ticket,total_amount
0,Fraud,107,114.519346,12253.57
1,Legit,99892,43.044731,4299824.27


### 📊 Interpretación de resultados: ¿cómo operan los fraudes?

Los datos pintan un patrón claro:

| Tipo | Transacciones | Ticket Promedio | Monto Total |
|------|----------------|------------------|-------------|
| **Fraud** | 107 | **$114.52** | 12,253.57 |
| **Legit** | 99,892 | $43.04 | 4,299,824.27 |

**Hallazgos clave:**

- Las transacciones fraudulentas son **casi 3 veces más altas en valor promedio** que las legítimas.
- Aunque solo representan **0.1%** del volumen, generan un impacto económico significativo.
- El fraude no destaca por cantidad, sino por **monto elevado**, lo que es típico en escenarios reales:  
  *los atacantes priorizan calidad sobre cantidad*.

Este tipo de análisis ayuda a diseñar reglas antifraude orientadas a:
- montos inusualmente altos,
- patrones de gasto atípicos por usuario,
- anomalías por categoría o ubicación.

Con estos indicadores, el modelo comienza a perfilar dónde se concentra el riesgo financiero.


## 📌 Análisis de fraude por categoría (MCC)

En esta sección profundizamos en qué categorías de comercio concentran más transacciones fraudulentas.
El objetivo es entender dónde se está filtrando el riesgo y si existen sectores particularmente vulnerables.

Para ello:

Estandarizamos los montos (amount_clean) eliminando símbolos y comas.

Unificamos los códigos MCC como enteros.

Cruzamos transactions con fraud_labels mediante un CTE para mantener el cálculo limpio.

Agrupamos por merchant_category_name y calculamos:

total_transactions

fraud_transactions

fraud_rate_pct: proporción de fraude por categoría

Filtramos solo categorías con al menos 1 transacción fraudulenta para no inflar falsos positivos.

Ordenamos de mayor a menor tasa de fraude.

El resultado revela patrones clave: algunas categorías poco frecuentes muestran una tasa de fraude desproporcionadamente alta, mientras que otras —aunque de gran volumen— concentran menos riesgo relativo.

In [21]:
query = """
WITH tx AS (
    SELECT 
        t.id,
        REPLACE(REPLACE(t.amount, '$', ''), ',', '') * 1.0 AS amount_clean,
        COALESCE(f.is_fraud, 0) AS is_fraud,
        CAST(t.mcc AS INTEGER) AS mcc_code
    FROM transactions t
    LEFT JOIN fraud_labels f
        ON t.id = f.transaction_id
)
SELECT
    m.merchant_category_name,
    COUNT(*) AS total_transactions,
    SUM(CASE WHEN tx.is_fraud = 1 THEN 1 ELSE 0 END) AS fraud_transactions,
    100.0 * SUM(CASE WHEN tx.is_fraud = 1 THEN 1 ELSE 0 END) / COUNT(*) AS fraud_rate_pct
FROM tx
JOIN mcc_codes m
    ON tx.mcc_code = m.mcc_code
GROUP BY m.merchant_category_name
HAVING fraud_transactions > 0
ORDER BY fraud_transactions DESC
LIMIT 10;
"""

df_fraud_mcc = pd.read_sql_query(query, conn)
display(df_fraud_mcc)


,merchant_category_name,total_transactions,fraud_transactions,fraud_rate_pct
0,Department Stores,3545,22,0.620592
1,Discount Stores,1778,12,0.674916
2,Wholesale Clubs,4542,9,0.198151
3,"Digital Goods - Media, Books, Apps",480,6,1.250000
4,Passenger Railways,145,5,3.448276
5,"Furniture, Home Furnishings, and Equipment Stores",35,5,14.285714
6,Electronics Stores,51,5,9.803922
7,Taxicabs and Limousines,3877,4,0.103173
8,"Lighting, Fixtures, Electrical Supplies",109,4,3.669725
9,Fast Food Restaurants,3761,4,0.106355


## 🧠 Interpretación principal

Department Stores y Discount Stores aparecen con volúmenes relativamente altos y una tasa de fraude significativa, lo que sugiere que el flujo de transacciones en estos sectores es atractivo para el fraude.

Categorías pequeñas como Furniture & Home Furnishings muestran tasas de fraude elevadísimas, pero con muy bajo volumen: esto debe interpretarse como señales puntuales, no tendencias globales.

Electronics Stores mantiene una tasa de fraude notablemente alta dado su volumen moderado, lo cual es coherente con la dinámica real: productos de alto valor → objetivo frecuente para fraudes.

## ⏰ 8. Análisis de fraude por hora del día

Para cerrar el análisis SQL, realizamos una evaluación por **hora del día**, con el objetivo de detectar ventanas temporales donde la actividad fraudulenta tiende a concentrarse.  
Este tipo de análisis es muy utilizado en **fintech, banca digital y prevención de fraude**, ya que permite:

- identificar patrones inusuales de comportamiento,
- ajustar reglas de riesgo basadas en horarios,
- detectar comportamientos automatizados (bots, scripts),
- validar si existe “fraude oportunista” cuando disminuye la supervisión humana.

### 🔍 ¿Qué hace esta consulta?

1. Extrae la hora (`%H`) de cada transacción.
2. Integra la etiqueta de fraude desde `fraud_labels`.
3. Calcula por hora:
   - número total de transacciones,
   - número de fraudes detectados,
   - tasa porcentual de fraude.
4. Ordena de mayor a menor tasa de fraude.

Este análisis revela **en qué momentos del día las transacciones fraudulentas tienden a ocurrir con mayor frecuencia relativa**, una pieza clave para generar alertas basadas en comportamiento.

---


In [22]:
query = query = """
WITH tx AS (
    SELECT 
        t.id,
        STRFTIME('%H', t.date) AS hour,
        COALESCE(f.is_fraud, 0) AS is_fraud
    FROM transactions t
    LEFT JOIN fraud_labels f
        ON t.id = f.transaction_id
)
SELECT 
    hour,
    COUNT(*) AS total_transactions,
    SUM(CASE WHEN is_fraud = 1 THEN 1 ELSE 0 END) AS fraud_transactions,
    100.0 * SUM(CASE WHEN is_fraud = 1 THEN 1 ELSE 0 END) / COUNT(*) AS fraud_rate_pct
FROM tx
GROUP BY hour
ORDER BY fraud_rate_pct DESC, hour;
"""

df_fraud_hours = pd.read_sql_query(query, conn)
display(df_fraud_hours)
"""
WITH tx AS (
    SELECT 
        t.id,
        STRFTIME('%H', t.date) AS hour,
        COALESCE(f.is_fraud, 0) AS is_fraud
    FROM transactions t
    LEFT JOIN fraud_labels f
        ON t.id = f.transaction_id
)
SELECT 
    hour,
    COUNT(*) AS total_transactions,
    SUM(CASE WHEN is_fraud = 1 THEN 1 ELSE 0 END) AS fraud_transactions,
    100.0 * SUM(CASE WHEN is_fraud = 1 THEN 1 ELSE 0 END) / COUNT(*) AS fraud_rate_pct
FROM tx
GROUP BY hour
ORDER BY fraud_rate_pct DESC, hour;
"""

df_fraud_hours = pd.read_sql_query(query, conn)
display(df_fraud_hours)


,hour,total_transactions,fraud_transactions,fraud_rate_pct
0,23,1172,10,0.853242
1,01,883,6,0.679502
2,02,850,4,0.470588
3,05,1421,5,0.351865
4,03,787,2,0.254130
5,22,3136,7,0.223214
6,21,3177,7,0.220334
7,19,3363,6,0.178412
8,06,5632,9,0.159801
9,20,3226,4,0.123993


,hour,total_transactions,fraud_transactions,fraud_rate_pct
0,23,1172,10,0.853242
1,01,883,6,0.679502
2,02,850,4,0.470588
3,05,1421,5,0.351865
4,03,787,2,0.254130
5,22,3136,7,0.223214
6,21,3177,7,0.220334
7,19,3363,6,0.178412
8,06,5632,9,0.159801
9,20,3226,4,0.123993


## 📊 Interpretación del comportamiento del fraude por hora del día

Los resultados muestran diferencias claras en el **riesgo relativo según la hora** en que ocurren las transacciones:

### 🔥 Horas más riesgosas (fraud_rate_pct más alto)
Las horas con mayor concentración proporcional de fraude fueron:

- **23:00 h** — picos atípicos con la tasa más alta.
- **01:00–02:00 h** — actividad elevada que puede sugerir automatización.
- **05:00 h** — otro horario potencialmente oportunista.

Este patrón es típico en sistemas donde el fraude:
- intenta evitar supervisión humana,
- se ejecuta mediante scripts nocturnos,
- o se aprovecha de baja actividad operacional.

### 🟦 Horas con riesgo moderado
Entre **19:00–22:00 h**, el fraude baja, pero sigue siendo superior al promedio.

### 🟩 Horas más seguras (menor incidencia relativa)
Las horas de **mayor actividad normal (08:00–17:00 h)** presentan tasas mínimas de fraude, lo que coincide con:

- mayor tráfico legítimo,
- mayor supervisión operativa,
- menor efectividad para ataques automatizados.

---

## 🎯 Insight clave

Aunque el **volumen** de fraude es bajo, su **concentración por hora** es reveladora:  
los defraudadores parecen aprovechar **ventanas nocturnas**, donde el tráfico legítimo es menor y pasar desapercibidos es más sencillo.

Este hallazgo permitiría a una fintech:

- reforzar reglas antifraude en horarios críticos,
- exigir validaciones adicionales (OTP, geolocación) durante la madrugada,
- ajustar modelos predictivos basados en timestamp.

---

## 🔐 Conclusión

La combinación de análisis transaccional, perfiles de usuario, categorías MCC y distribución horaria construye una visión sólida del **riesgo operativo**.  
Con ello, la empresa podría diseñar **estrategias proactivas de mitigación de fraude**, basadas en evidencia real y patrones observados.

---


# 🧠 9. Síntesis de hallazgos e insights financieros

A partir del análisis SQL realizado sobre las tablas de usuarios, tarjetas, transacciones, categorías MCC y etiquetas de fraude, podemos extraer una visión integral del comportamiento financiero dentro de la fintech.  
Los hallazgos permiten comprender **cómo se mueve el dinero, cómo se comportan los usuarios y dónde se encuentran los riesgos operativos más importantes**.

---

## 📊 1. Volumen y comportamiento transaccional

- El dataset contiene **99,999 transacciones**, suficiente para observar patrones robustos.  
- El tráfico se mantiene relativamente estable día a día, sin picos atípicos que sugieran estacionalidades fuertes.  
- El **ticket promedio general** es de aproximadamente **\$43.12**, lo que posiciona a la fintech en un segmento de pagos cotidianos, de bajo y mediano monto.

**Insight:**  
El negocio parece orientado al consumo recurrente y a transacciones masivas de bajo valor, por lo que la eficiencia y velocidad del procesamiento son factores estratégicos.

---

## 🛍️ 2. Categorías MCC: ¿en qué gastan los usuarios?

Las categorías con mayor volumen de transacciones fueron:

1. **Grocery Stores / Supermarkets**  
2. **Miscellaneous Food Stores**  
3. **Service Stations**  
4. **Restaurants y Fast Food**  

**Patrón dominante:**  
El gasto se concentra en **consumo básico y usos cotidianos**, lo que refuerza la idea de que la fintech opera como medio de pago diario.

**Ticket promedio por categoría:**  
Las categorías de mayor valor promedio fueron:

- **Money Transfer**  
- **Wholesale Clubs**  
- **Department Stores**

Estas categorías —aunque no siempre tienen el mayor volumen— sí concentran **mayor monto por transacción**, lo que las vuelve relevantes desde perspectiva de margen.

---

## 👥 3. Comportamiento por grupo de edad

El análisis muestra una relación clara:  
**A mayor edad, mayor ticket promedio.**

- 18–29 años → ticket promedio más bajo (≈ \$30)  
- 50–59 años → ticket promedio más alto (≈ \$44.5)  
- 60+ años → consumo también elevado y estable  

**Interpretación:**  
Los usuarios mayores tienen mayor capacidad de gasto y utilizan los servicios para transacciones de mayor valor. Esto abre puertas a productos premium o líneas de crédito dirigidas a este segmento.

---

## 🔁 4. Recurrencia de usuarios

Algunos usuarios acumulan **más de 300 transacciones individuales**, lo cual indica:

- un grupo de usuarios “heavy usage” con alta fidelidad,
- potencial para estrategias de retención y beneficios segmentados,
- insights valiosos para modelos de CLTV.

**Insight:**  
No todos los usuarios aportan el mismo valor. Hay un subconjunto altamente activo que podría impulsar programas “Top User” o cashback inteligente.

---

## 🔐 5. Exploración y patrones de fraude

La proporción de fraude encontrada en la muestra es:

- **107 transacciones fraudulentas**  
- **Fraud Rate ≈ 0.107%**

Aunque el volumen es pequeño, se detectan patrones claros:

### 🔎 Categorías con mayor fraude relativo
Las tasas más altas de fraude se concentran en:

- **Furniture / Equipment Stores**  
- **Electronics Stores**  
- **Digital Goods – Apps / Media**  
- **Passenger Railways**  
- **Discount Stores**  

**Interpretación:**  
Son categorías donde:
- es fácil revender bienes,
- existen transacciones rápidas/no presenciales,
- hay menos fricción operativa.

Estos sectores suelen estar entre los más atacados en cualquier ecosistema de pagos.

### 🕒 Horarios de mayor riesgo
El fraude se concentra desproporcionadamente en:

- **23:00 h** (pico máximo)  
- **01:00–02:00 h**  
- **05:00 h**  

El patrón sugiere actividad automatizada o intentos de aprovechar periodos de menor supervisión humana.

**Insight clave:**  
El fraude no ocurre al azar; sigue ventanas de oportunidad específicas. Esto permitiría activar **reglas dinámicas** como:
- bloqueo de alto riesgo nocturno,
- doble validación en horarios críticos,
- modelos ML que incluyan timestamp como feature relevante.

---

## 🧩 Conclusión general

El análisis SQL permite construir una radiografía clara del ecosistema transaccional:

- La fintech opera principalmente como medio de pago cotidiano.  
- Los usuarios mayores impulsan transacciones de mayor valor.  
- Las categorías de consumo diario dominan el volumen.  
- Existen segmentos de usuarios altamente recurrentes que representan oportunidades de fidelización.  
- El fraude es bajo en volumen, pero muy claro en patrones: categorías específicas y horas nocturnas.

Este tipo de análisis no solo revela lo que ocurre, sino **cómo puede la empresa actuar**: optimizar prevención de fraude, ajustar estrategias de producto, mejorar retención y diseñar soluciones segmentadas con base en datos reales.

---

